## **Neural network - Recognizing hand-written digits**

Participants are expected to code up a neural network based on Chapter 1 of Michael Nielsen's book, specifically working with the MNIST dataset. This assignment should be completed during the class, and participants are required to submit their working notebooks.

Link to chapter 1: http://neuralnetworksanddeeplearning.com/chap1.html

In [1]:
# Import all required libraries
import random
import numpy as np

In [2]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

The above cell defines two functions:

1. **`sigmoid(z)`**: This function implements the sigmoid activation function, which is commonly used in neural networks. It takes an input `z` and returns the sigmoid value \( \frac{1}{1 + e^{-z}} \).

2. **`sigmoid_prime(z)`**: This function computes the derivative of the sigmoid function. It is used in backpropagation during the training of neural networks to update the weights and biases. The derivative is calculated as \( \sigma(z) \cdot (1 - \sigma(z)) \), where \( \sigma(z) \) is the sigmoid function.


In [8]:
class NeuralNetwork(object):
    def __init__(self, layer_sizes):
        self.num_layers = len(layer_sizes)
        self.layer_sizes = layer_sizes
        self.biases = [np.random.randn(y, 1) for y in layer_sizes[1:]] #Starting from the second layer, we have biases since the first layer is the input layer
        self.weights = [np.random.randn(y, x) for x, y in zip(layer_sizes[:-1], layer_sizes[1:])] #Gaussian distribution with mean 0 and variance 1 for weights

    def feedforward(self, input_data):
        for b, w in zip(self.biases, self.weights):
            input_data = sigmoid(np.dot(w, input_data) + b)
        return input_data

    def train(self, training_data, epochs, batch_size, learning_rate, test_data=None):
        if test_data: 
            test_size = len(test_data)
        train_size = len(training_data)
        for epoch in range(epochs):
            random.shuffle(training_data)
            # Mini-batch stochastic gradient descent
            batches = [training_data[k:k + batch_size] for k in range(0, train_size, batch_size)]
            for batch in batches:
                self.update_batch(batch, learning_rate)
            if test_data:
                print(f"Epoch {epoch}: {self.evaluate(test_data)} / {test_size}")
            else:
                print(f"Epoch {epoch} complete")

    def update_batch(self, batch, learning_rate):
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in batch:
            delta_grad_b, delta_grad_w = self.backprop(x, y)
            grad_b = [gb + dgb for gb, dgb in zip(grad_b, delta_grad_b)]
            grad_w = [gw + dgw for gw, dgw in zip(grad_w, delta_grad_w)]
        self.weights = [w - (learning_rate / len(batch)) * gw for w, gw in zip(self.weights, grad_w)]
        self.biases = [b - (learning_rate / len(batch)) * gb for b, gb in zip(self.biases, grad_b)]

    def backprop(self, x, y):
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        #feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            grad_b[-l] = delta
            grad_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        return grad_b, grad_w

    def evaluate(self, test_data):
        #output is assumed to be the index of whichever neuron in the final layer has the highest activation
        results = [(np.argmax(self.feedforward(x)), y) for x, y in test_data]
        return sum(int(x == y) for x, y in results)

    def cost_derivative(self, output_activations, y):
        return output_activations - y

The above cell defines the `NeuralNetwork` class, which implements a multi-layer neural network with the following key components:

1. **`__init__(self, layer_sizes)`**: Initializes the neural network with a given architecture (list of layer sizes). It randomly initializes the biases and weights for the network. Biases are initialized for all layers except the input layer, and weights are initialized using a Gaussian distribution.

2. **`feedforward(self, input_data)`**: This method takes an input vector and propagates it forward through the network, layer by layer, applying the activation function (sigmoid) at each step.

3. **`train(self, training_data, epochs, batch_size, learning_rate, test_data=None)`**: This method trains the network using mini-batch stochastic gradient descent (SGD). It takes the training data, number of epochs, batch size, and learning rate as input. If test data is provided, the network’s performance is evaluated after each epoch.

4. **`update_batch(self, batch, learning_rate)`**: Updates the network's weights and biases by performing gradient descent on the given mini-batch. The gradients are calculated using backpropagation and applied to update the parameters.

5. **`backprop(self, x, y)`**: This method performs backpropagation for a single training example `(x, y)`, calculating the gradients of the cost function with respect to the weights and biases. It performs the forward pass, then computes the gradients during the backward pass.

6. **`evaluate(self, test_data)`**: Evaluates the network's performance on the test data by comparing the predicted output (the neuron with the highest activation in the output layer) with the true labels. It returns the number of correct predictions.

7. **`cost_derivative(self, output_activations, y)`**: Calculates the derivative of the cost function with respect to the output activations. This is used in backpropagation to compute the gradients.


In [15]:
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('Data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))  # Convert zip to list
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))  # Convert zip to list
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))  # Convert zip to list
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


The above cell defines functions for loading and processing the MNIST dataset:

1. **`load_data()`**: This function loads the MNIST dataset from a gzip-compressed pickle file (`mnist.pkl.gz`). It returns the dataset as three separate tuples: `training_data`, `validation_data`, and `test_data`.

2. **`load_data_wrapper()`**: This function processes the raw MNIST data into a format more suitable for training a neural network. It reshapes the image data into 784-dimensional column vectors and converts the label data into a 10-dimensional one-hot encoded vector using the `vectorized_result()` function. It returns the data in the form of lists of 2-tuples: `(input, label)`.

3. **`vectorized_result(j)`**: This function creates a 10-dimensional one-hot encoded vector for the label `j` (where `j` is the digit class, from 0 to 9). The vector has a val


In [16]:
training_data, validation_data, test_data = load_data_wrapper()

In [ ]:
net = NeuralNetwork([784, 30, 10])
net.train(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8219 / 10000
Epoch 1: 8294 / 10000
Epoch 2: 8438 / 10000
Epoch 3: 9312 / 10000
Epoch 4: 9360 / 10000
Epoch 5: 9381 / 10000
Epoch 6: 9404 / 10000
Epoch 7: 9428 / 10000
Epoch 8: 9445 / 10000
Epoch 9: 9475 / 10000
Epoch 10: 9457 / 10000
Epoch 11: 9470 / 10000
Epoch 12: 9481 / 10000
Epoch 13: 9490 / 10000
Epoch 14: 9468 / 10000
Epoch 15: 9500 / 10000
Epoch 16: 9488 / 10000
Epoch 17: 9503 / 10000
Epoch 18: 9498 / 10000
Epoch 19: 9510 / 10000
Epoch 20: 9491 / 10000
Epoch 21: 9477 / 10000
Epoch 22: 9496 / 10000
Epoch 23: 9505 / 10000
Epoch 24: 9512 / 10000
Epoch 25: 9486 / 10000
Epoch 26: 9496 / 10000
Epoch 27: 9498 / 10000
Epoch 28: 9506 / 10000
Epoch 29: 9507 / 10000


We see the output of training the neural network over 30 epochs. After each epoch, the network's performance on the test data is printed, showing the number of correct predictions out of 10,000 test examples. 

The format of the output is as follows:
- **Epoch X**: The current epoch number.
- **Y / 10000**: The number of correct predictions (Y) out of 10,000 test examples.

As the epochs progress, the number of correct predictions generally increases, indicating that the network is learning and improving its performance on the test set. We see that we obtained an accuracy of 95.07%.


In [20]:
net2 = NeuralNetwork([784, 50, 10])
net2.train(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8322 / 10000
Epoch 1: 9277 / 10000
Epoch 2: 9286 / 10000
Epoch 3: 9399 / 10000
Epoch 4: 9383 / 10000
Epoch 5: 9467 / 10000
Epoch 6: 9494 / 10000
Epoch 7: 9506 / 10000
Epoch 8: 9514 / 10000
Epoch 9: 9534 / 10000
Epoch 10: 9547 / 10000
Epoch 11: 9546 / 10000
Epoch 12: 9544 / 10000
Epoch 13: 9558 / 10000
Epoch 14: 9570 / 10000
Epoch 15: 9570 / 10000
Epoch 16: 9543 / 10000
Epoch 17: 9570 / 10000
Epoch 18: 9571 / 10000
Epoch 19: 9579 / 10000
Epoch 20: 9565 / 10000
Epoch 21: 9572 / 10000
Epoch 22: 9564 / 10000
Epoch 23: 9569 / 10000
Epoch 24: 9584 / 10000
Epoch 25: 9575 / 10000
Epoch 26: 9578 / 10000
Epoch 27: 9572 / 10000
Epoch 28: 9562 / 10000
Epoch 29: 9585 / 10000


When we increase the hidden layer neurons to 50 from 30, we see that our accuracy increaes to 95.85%. 